In [ ]:
import pandas as pd
import pingouin as pg
import numpy as np
from math import ceil
from scipy.stats import binom_test
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from glob import glob
import os
from plotting.plot import pretty_behav_plot

%load_ext autoreload
%autoreload 2

## Behavioral data

In [ ]:
allsubjdirs = sorted(glob('../../bids/sub-*'))
subjlist = [f'sub-{s:03d}' for s in range(1, 36)] #if s != 6]
allsubjs_trials = pd.DataFrame()
for i, s in enumerate(subjlist):
    behfile = glob(os.path.join('../../bids', s, 'func', '*beh.tsv'))[0]
    behav = pd.read_csv(behfile, sep='\t')
    behav.insert(0, 'Subject', i + 1)
    allsubjs_trials = pd.concat([allsubjs_trials, behav])

In [ ]:
allsubjs_trials

## Plot staircases

In [ ]:
n_missed = []
remove_subjs = []
fig = plt.figure(figsize=(15,10))
nsubjs = len(allsubjs_trials.Subject.unique())
for s in allsubjs_trials.Subject.unique():
    thisstair = np.array(abs(allsubjs_trials.loc[allsubjs_trials['Subject'] == s]['Diff']))
    realstair = abs(allsubjs_trials[allsubjs_trials['Subject']==s].Orients_1.values - 
                   allsubjs_trials[allsubjs_trials['Subject']==s].Orients_2.values)
    theseresps = np.array(allsubjs_trials.loc[allsubjs_trials['Subject'] == s]['Hit'])
    howmuchceil = sum(realstair==10)
    missingtrials = np.count_nonzero(np.isnan(theseresps))
    n_missed.append(missingtrials)
    ax = fig.add_subplot(ceil(nsubjs/4), 4, s)
    ax.set_ylim([0, 25])
    k = np.nansum(theseresps)
    n = len(thisstair)
    if binom_test(k, n, p=0.5, alternative='greater')>0.05:
    #if howmuchceil >= len(thisstair)/4 or missingtrials >= len(thisstair)/3:
        #print(k/n)
        ax.plot(thisstair, 'r')
        remove_subjs.append(s)
    else:
        ax.plot(thisstair, 'k')
    ax.plot(realstair, 'b')

In [ ]:
# Which participants to remove
remove_subjs

In [ ]:
#allsubjs_trials = allsubjs_trials[~allsubjs_trials['Subject'].isin(remove_subjs)]

## Expected vs Unexpected t-test

In [ ]:
allsubj_means = allsubjs_trials.groupby(['Subject', 'Consistent']).mean().reset_index()

In [ ]:
allsubj_means.groupby(['Consistent'])['Hit'].agg(['mean', 'sem'])

In [ ]:
pg.ttest(allsubj_means[allsubj_means['Consistent']==1]['Hit'], 
         allsubj_means[allsubj_means['Consistent']==0]['Hit'], paired=True)

## Plot

In [ ]:
pretty_behav_plot(allsubj_means, saveimg=True)

In [ ]:
# Get all differences
alldiffs = []
for sub in allsubj_means.Subject.unique():
    thisdiff = allsubj_means[(allsubj_means.Subject==sub)&(allsubj_means.Consistent==1)].Hit.values[0]-\
               allsubj_means[(allsubj_means.Subject==sub)&(allsubj_means.Consistent==0)].Hit.values[0]
    alldiffs.append({'Subject': sub, 'Difference': thisdiff})
alldiffs = pd.DataFrame(alldiffs)

In [ ]:
fig = plt.figure(figsize=(10,8))
gs = GridSpec(1, 2, figure=fig)
with sns.axes_style('white'):
    #ax0 = fig.add_subplot(121)
    ax0 = fig.add_subplot(gs[0, :-1])
    sns.barplot(x='Consistent', y='Hit', data=allsubj_means, ci=68, order=[1.0, 0.0], palette='Set2', ax=ax0)
    ax0.set_ylabel('Hit rate', fontsize=24)
    plt.yticks(fontsize=20) 
    ax0.set_xlabel(None)
    ax0.set_xticklabels(['Expected', 'Unexpected'], fontsize=20)
    ax0.set(ylim=(0.5, 0.75))
    ax0.spines['top'].set_visible(False)
    ax0.spines['right'].set_visible(False)
with sns.axes_style('white'):
    #ax1 = fig.add_subplot(122)
    ax1 = fig.add_subplot(gs[0, -1])
    sns.violinplot(y='Difference', data=alldiffs, color=".8", inner=None)
    sns.stripplot(y='Difference', data=alldiffs, jitter=0.07, ax=ax1, color='black', alpha=.5)
    # Get mean and 95% CI:
    meandiff = alldiffs['Difference'].mean()
    tstats = pg.ttest(alldiffs['Difference'], 0.0)
    ci95 = tstats['CI95%'][0]
    #plt.gca().set_aspect('equal')
    #ax1.axis('equal')
    for tick in ax1.get_xticks():
        ax1.plot([tick-0.1, tick+0.1], [meandiff, meandiff],
                    lw=4, color='k')
        ax1.plot([tick, tick], [ci95[0], ci95[1]], lw=3, color='k')
        ax1.plot([tick-0.01, tick+0.01], [ci95[0], ci95[0]], lw=3, color='k')
        ax1.plot([tick-0.01, tick+0.01], [ci95[1], ci95[1]], lw=3, color='k')
        #circlemarker = matplotlib.patches.Ellipse((tick+0.0032, meandiff), 0.06, 0.02, color='k')
        #ax1.add_patch(circlemarker)
    ax1.axhline(0.0, linestyle='--', color='black')
    plt.yticks(fontsize=20) 
    ax1.set_ylabel('Δ Hit rate', fontsize=24)
    ax1.set(ylim=(-0.2, 0.4))
    ax1.axes_style = 'white'
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['bottom'].set_visible(False)
plt.tight_layout()
#plt.savefig('Plots/Behav_acc.pdf')

## Demographic information

In [ ]:
particfile = '../../bids/participants.tsv'
particinfo = pd.read_csv(particfile, sep='\t')

In [ ]:
#particinfo = particinfo[particinfo.participant_id!='sub-006']

In [ ]:
particinfo.age.agg(['mean', 'std'])

In [ ]:
particinfo.sex.value_counts()